In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# grab NYT githhub data
url_county = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
url_state = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv'
url_country = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv'

In [3]:
county_data = pd.read_csv(url_county)
state_data = pd.read_csv(url_state)

In [4]:
county_data = county_data[~county_data['state'].isin(['District of Columbia','Northern Mariana Islands','Guam','Puerto Rico','Virgin Islands'])].reset_index(drop=True)
state_data = state_data[~state_data['state'].isin(['District of Columbia','Northern Mariana Islands','Guam','Puerto Rico','Virgin Islands'])].reset_index(drop=True)

In [5]:
county_data=county_data.dropna(subset=['county'])


In [6]:
def classification_average_cases_change(updown,x):
    if updown=='decrease' and x>=15:
        return 'falling'
    
    elif updown=='decrease' and x<15:
        return 'about the same'
    
    elif updown=='change':
        return 'about the same'
    
    elif updown=='increase' and x<50:
        return 'about the same'
    
    elif updown =='increase':
        if x >=50 and x<200:
            return 'great'
        if x >=200 and x<300:
            return 'greater'
        if x >=300:
            return 'greatest'
    else:
        return

In [7]:
def gather_data_state(data,loc):
    print(loc)
#     daily_cases = pd.DataFrame()
#     daily_deaths = pd.DataFrame()
    totals = pd.DataFrame()
    present_day = []
    
    
    if len(data['cases'])>14:

        daily_cases = np.zeros(len(data['cases']))
        daily_cases[0] = data['cases'].iloc[0]
        
        daily_deaths = np.zeros(len(data['deaths']))
        daily_deaths[0] = data['deaths'].iloc[0]

        # for loop checks to see if data reports a negative daily cases number.  If it does, the number is recorded as zero instead
        for idx in np.arange(1,len(data['cases'])):
            daily_cases[idx] = max(data['cases'].iloc[idx]-data['cases'].iloc[idx-1],0)
            daily_deaths[idx] = max(data['deaths'].iloc[idx]-data['deaths'].iloc[idx-1],0)
            
        daily_cases=pd.DataFrame(daily_cases,columns=['daily_cases'])
        daily_deaths=pd.DataFrame(daily_deaths,columns=['daily_deaths'])
        # calculate averages
        sevenMA = daily_cases['daily_cases'].rolling(window=7).mean()
        sevenMAdeath = daily_deaths['daily_deaths'].rolling(window=7).mean()
        sevenMA1diff = sevenMA.diff(periods=6)
        sevenMA2diff = sevenMA.diff(periods=13)
        twoweekMA = daily_cases['daily_cases'].rolling(window=14).mean()
        twoweekMAdiff = twoweekMA.diff(periods=13)
        cumsum_cases=daily_cases['daily_cases'].cumsum()
        cumsum_deaths=daily_deaths['daily_deaths'].cumsum()
        sevenMAcumsum= state['cases'].rolling(window=7).mean()
        data['sevenMA'] =sevenMA
        data['sevenMA1diff']= sevenMA1diff
        data['sevenMA2diff']= sevenMA2diff
        data['twoweekMA']=twoweekMA
        data['twoweekMAdiff']=twoweekMAdiff
        data['sevenMAcumsum']=sevenMAcumsum
        data['daily_cases']=daily_cases['daily_cases'].values
        data['daily_deaths']=daily_deaths['daily_deaths'].values
        data['cumsum_cases']=cumsum_cases
        data['cumsum_deaths']=cumsum_deaths
        data['sevenMAdeaths']=sevenMAdeath
        totals = totals.append(data)

        # gather information
        present_day.append((data['date'].iloc[-1],
                         data['state'].iloc[-1],
                         cumsum_cases.iloc[-1],
                         cumsum_deaths.iloc[-1],
                         daily_cases['daily_cases'].iloc[-1],
                         sevenMA.iloc[-1],
                         sevenMA.iloc[-7],
                         sevenMA.iloc[-14],
                         sevenMA1diff.iloc[-1],
                         sevenMA2diff.iloc[-1],
                         twoweekMA.iloc[-1],
                         twoweekMA.iloc[-14],
                         twoweekMAdiff.iloc[-1]
                        ))   
    

        info=pd.DataFrame(present_day,columns=['date','state','cumsum_cases','cumsum_deaths','daily_cases','sevenMA','sevenMA1wktrailing',
                                            'sevenMA2wktrailing','sevenMA1wkdiff','sevenMA2wkdiff',
                                            'twoweekMA','twoweekMAtrailing','twoweekMAdiff'])

        # creating categorical values based on averages
        info['avg_7_1wk_change_dir']=info['sevenMA1wkdiff'].map(lambda x: 'increase' if x>0 else('change' if x==0 else 'decrease'))
        info['avg_7_2wk_change_dir']=info['sevenMA2wkdiff'].map(lambda x: 'increase' if x>0 else('change' if x==0 else 'decrease'))
        # address dividing by zero
        # counties['sevenMA1wktrailing']=counties['sevenMA1wktrailing'].replace(0,1e-6)
        # counties['sevenMA2wktrailing']=counties['sevenMA2wktrailing'].replace(0,1e-6)

        info['avg_14_change_dir']=info['twoweekMAdiff'].map(lambda x: 'increase' if x>0 else('change' if x==0 else 'decrease'))
        # counties['twoweekMAtrailing']=counties['twoweekMAtrailing'].replace(0,1e-6)



        info['avg_7_1wk_pct_change']= 100*info['sevenMA1wkdiff'].abs() / info['sevenMA1wktrailing']
        info['avg_7_2wk_pct_change']= 100*info['sevenMA2wkdiff'].abs() / info['sevenMA2wktrailing']
        info['avg_14_pct_change']= 100*info['twoweekMAdiff'].abs() / info['twoweekMAtrailing']

        info['case7_1wk_change'] = info.apply(lambda x: classification_average_cases_change(x['avg_7_1wk_change_dir'], x['avg_7_1wk_pct_change']), axis=1)
        info['case7_2wk_change'] = info.apply(lambda x: classification_average_cases_change(x['avg_7_2wk_change_dir'], x['avg_7_2wk_pct_change']), axis=1)
        info['case14_change'] = info.apply(lambda x: classification_average_cases_change(x['avg_14_change_dir'], x['avg_14_pct_change']), axis=1)

        return totals, info
    
    else:
        print('pass')

        return totals, totals


all_state_totals=pd.DataFrame()
all_states_present_day=pd.DataFrame()
i=0
for state_loc in state_data['state'].unique():
    i=i+1
    state=pd.DataFrame(state_data.loc[state_data['state']==state_loc].sort_values(['date'], ascending=True)).reset_index(drop=True)
    state_totals, states_present_day = gather_data_state(state,state_loc)
    all_state_totals = all_state_totals.append(state_totals)
    all_states_present_day=all_states_present_day.append(states_present_day)
    

all_states_present_day=all_states_present_day.sort_values(by=['cumsum_cases'],ascending=False).reset_index(drop=True)
all_states_present_day=all_states_present_day.reset_index()
all_states_present_day['index']=all_states_present_day['index']+1


all_state_totals.to_csv('state_totals.csv',index=False)
all_states_present_day.to_csv('states_present_day.csv',index=False)

Washington
Illinois
California
Arizona
Massachusetts
Wisconsin
Texas
Nebraska
Utah
Oregon
Florida
New York
Rhode Island
Georgia
New Hampshire
North Carolina
New Jersey
Colorado
Maryland
Nevada
Tennessee
Hawaii
Indiana
Kentucky
Minnesota
Oklahoma
Pennsylvania
South Carolina
Kansas
Missouri
Vermont
Virginia
Connecticut
Iowa
Louisiana
Ohio
Michigan
South Dakota
Arkansas
Delaware
Mississippi
New Mexico
North Dakota
Wyoming
Alaska
Maine
Alabama
Idaho
Montana
West Virginia


In [8]:
def gather_data_county(data,loc):
#     print(loc)
    daily_cases = pd.DataFrame()
    totals = pd.DataFrame()
    present_day = []
    
    
    if len(data['cases'])>14:

        daily_cases = np.zeros(len(data['cases']))
        daily_cases[0] = data['cases'].iloc[0]

        # for loop checks to see if data reports a negative daily cases number.  If it does, the number is recorded as zero instead
        for idx in np.arange(1,len(data['cases'])):
            daily_cases[idx] = max(data['cases'].iloc[idx]-data['cases'].iloc[idx-1],0)
            
        daily_cases=pd.DataFrame(daily_cases,columns=['daily_cases'])
        # calculate averages
        sevenMA = daily_cases['daily_cases'].rolling(window=7).mean()
        sevenMA1diff = sevenMA.diff(periods=6)
        sevenMA2diff = sevenMA.diff(periods=13)
        twoweekMA = daily_cases['daily_cases'].rolling(window=14).mean()
        twoweekMAdiff = twoweekMA.diff(periods=13)
        cumsum=daily_cases['daily_cases'].cumsum()
        sevenMAcumsum= state['cases'].rolling(window=7).mean()
        data['sevenMA'] =sevenMA
        data['sevenMA1diff']= sevenMA1diff
        data['sevenMA2diff']= sevenMA2diff
        data['twoweekMA']=twoweekMA
        data['twoweekMAdiff']=twoweekMAdiff
        data['sevenMAcumsum']=sevenMAcumsum
        data['daily_cases']=daily_cases['daily_cases'].values
        data['cumsum']=cumsum
        totals = totals.append(data)

        # gather information
        present_day.append((data['date'].iloc[-1],
                         data['state'].iloc[-1],
                         loc,
                         cumsum.iloc[-1],
                         daily_cases['daily_cases'].iloc[-1],
                         sevenMA.iloc[-1],
                         sevenMA.iloc[-7],
                         sevenMA.iloc[-14],
                         sevenMA1diff.iloc[-1],
                         sevenMA2diff.iloc[-1],
                         twoweekMA.iloc[-1],
                         twoweekMA.iloc[-14],
                         twoweekMAdiff.iloc[-1]
                        ))   
    

        info=pd.DataFrame(present_day,columns=['date','state','county','cumsum','daily_cases','sevenMA','sevenMA1wktrailing',
                                            'sevenMA2wktrailing','sevenMA1wkdiff','sevenMA2wkdiff',
                                            'twoweekMA','twoweekMAtrailing','twoweekMAdiff'])

        # creating categorical values based on averages
        info['avg_7_1wk_change_dir']=info['sevenMA1wkdiff'].map(lambda x: 'increase' if x>0 else('change' if x==0 else 'decrease'))
        info['avg_7_2wk_change_dir']=info['sevenMA2wkdiff'].map(lambda x: 'increase' if x>0 else('change' if x==0 else 'decrease'))
        # address dividing by zero
        # counties['sevenMA1wktrailing']=counties['sevenMA1wktrailing'].replace(0,1e-6)
        # counties['sevenMA2wktrailing']=counties['sevenMA2wktrailing'].replace(0,1e-6)

        info['avg_14_change_dir']=info['twoweekMAdiff'].map(lambda x: 'increase' if x>0 else('change' if x==0 else 'decrease'))
        # counties['twoweekMAtrailing']=counties['twoweekMAtrailing'].replace(0,1e-6)



        info['avg_7_1wk_pct_change']= 100*info['sevenMA1wkdiff'].abs() / info['sevenMA1wktrailing']
        info['avg_7_2wk_pct_change']= 100*info['sevenMA2wkdiff'].abs() / info['sevenMA2wktrailing']
        info['avg_14_pct_change']= 100*info['twoweekMAdiff'].abs() / info['twoweekMAtrailing']

        info['case7_1wk_change'] = info.apply(lambda x: classification_average_cases_change(x['avg_7_1wk_change_dir'], x['avg_7_1wk_pct_change']), axis=1)
        info['case7_2wk_change'] = info.apply(lambda x: classification_average_cases_change(x['avg_7_2wk_change_dir'], x['avg_7_2wk_pct_change']), axis=1)
        info['case14_change'] = info.apply(lambda x: classification_average_cases_change(x['avg_14_change_dir'], x['avg_14_pct_change']), axis=1)
        return totals, info


    else:
#         print('pass')
        return totals,totals
        



all_county_totals=pd.DataFrame()
all_county_present_day=pd.DataFrame()
i=0
for state_loc in county_data['state'].unique():
    i=i+1
    print(i,'+',state_loc)
    state=pd.DataFrame(county_data.loc[county_data['state']==state_loc].sort_values(['date'], ascending=True)).reset_index(drop=True)
    for county_loc in state['county'].unique():
        county=pd.DataFrame(state.loc[state['county']==county_loc].sort_values(['date'], ascending=True)).reset_index(drop=True)
        county_totals, county_present_day = gather_data_county(county,county_loc)
        all_county_totals = all_county_totals.append(county_totals)
        all_county_present_day=all_county_present_day.append(county_present_day)

all_county_totals.to_csv('county_totals.csv',index=False)
all_county_present_day.to_csv('counties_present_day.csv',index=False)

1 + Washington
2 + Illinois
3 + California
4 + Arizona
5 + Massachusetts
6 + Wisconsin
7 + Texas
8 + Nebraska
9 + Utah
10 + Oregon
11 + Florida
12 + New York
13 + Rhode Island
14 + Georgia
15 + New Hampshire
16 + North Carolina
17 + New Jersey
18 + Colorado
19 + Maryland
20 + Nevada
21 + Tennessee
22 + Hawaii
23 + Indiana
24 + Kentucky
25 + Minnesota
26 + Oklahoma
27 + Pennsylvania
28 + South Carolina
29 + Kansas
30 + Missouri
31 + Vermont
32 + Virginia
33 + Connecticut
34 + Iowa
35 + Louisiana
36 + Ohio
37 + Michigan
38 + South Dakota
39 + Arkansas
40 + Delaware
41 + Mississippi
42 + New Mexico
43 + North Dakota
44 + Wyoming
45 + Alaska
46 + Maine
47 + Alabama
48 + Idaho
49 + Montana
50 + West Virginia
